Задача:

Производственная система состоит из m станков,каждый из которых подвержен случайным поломкам.  Перед поломкой станок работает в течение промежутка времени, который представляет собой экспоненциально распределённую случайную величину со средним значением 8 часов.  Ремонтом станков занимается s ремонтников(где s — фиксированное положительное целое число). Для того, чтобы починить один станок, одному ремонтнику требуется экспоненциально распределенный промежуток времени со средним значением 2 часа; Ремонтом сломанного станка может заниматься только один ремонтник, даже если в это время есть другие свободные ремонтники. Если одновременно ломается более s станков, они формируют очередь на ремонт с дисциплиной FIFO и ждут первого свободного ремонтника.  Далее ремонтник чинит сломанный станок до тех пор, пока он не будет починен, независимо от того, что еще происходит в системе.  Предположим, что каждый час поломки одного станка стоит 50 долларов, а час работы ремонтника стоит 10 долларов.  (Ремонтникам платят почасовую оплату независимо от того, работают ли они на самом деле.) 

Предположим, что количество станков в системе равно m = 5, но напишите общий код, позволяющий учитывать значение m, достигающее 20, путем изменения входного параметра.

Смоделируйте систему ровно при 800 часах работы для каждой стратегии занятости ( s = 1, 2, 3, 4, 5 ) ,  чтобы определить, какая из них приводит к наименьшим ожидаемым средним затратам в час, допустив, что в момент времени 0 все машины только что были отремонтированы.

In [19]:
import numpy as np

def exponential_distribution(mean, size):
    return np.random.exponential(mean, size)

def simulate_system(m, s, hours):
    machine_working_time = exponential_distribution(8, m)
    machine_breakdown_time = np.zeros(m)
    total_cost = 0

    for hour in range(hours):
        for i in range(m):
            if machine_breakdown_time[i] == 0:
                machine_working_time[i] -= 1
                if machine_working_time[i] <= 0:
                    machine_breakdown_time[i] = exponential_distribution(8, 1)
            else:
                machine_breakdown_time[i] -= 1
                if machine_breakdown_time[i] <= 0:
                    machine_breakdown_time[i] = 0
                    machine_working_time[i] = exponential_distribution(8, 1)
                    total_cost += 50

        repair_queue = 0
        for i in range(m):
            if machine_breakdown_time[i] > 0:
                repair_queue += 1

        while repair_queue > 0:
            repair_time = exponential_distribution(2, 1)
            total_cost += s * 10
            repair_queue -= s
            if repair_queue < 0:
                total_cost -= (abs(repair_queue) * 10)

    return total_cost / hours


m_values = range(1, 21, 1)
s_values = range (1, 6, 1)

for m in m_values:
    for s in s_values:
        avg_cost = simulate_system(m, s, 800)
        print(f"For m = {m} and s = {s}, average cost per hour is: {avg_cost}")

/tmp/ipykernel_42271/3750290773.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_breakdown_time[i] = exponential_distribution(8, 1)
/tmp/ipykernel_42271/3750290773.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_working_time[i] = exponential_distribution(8, 1)


For m = 1 and s = 1, average cost per hour is: 8.0875
For m = 1 and s = 2, average cost per hour is: 8.7625
For m = 1 and s = 3, average cost per hour is: 7.3375
For m = 1 and s = 4, average cost per hour is: 8.3625
For m = 1 and s = 5, average cost per hour is: 8.1875
For m = 2 and s = 1, average cost per hour is: 16.0625
For m = 2 and s = 2, average cost per hour is: 15.5
For m = 2 and s = 3, average cost per hour is: 15.875
For m = 2 and s = 4, average cost per hour is: 15.2
For m = 2 and s = 5, average cost per hour is: 15.7
For m = 3 and s = 1, average cost per hour is: 23.85
For m = 3 and s = 2, average cost per hour is: 22.975
For m = 3 and s = 3, average cost per hour is: 23.9375
For m = 3 and s = 4, average cost per hour is: 22.9375
For m = 3 and s = 5, average cost per hour is: 23.775
For m = 4 and s = 1, average cost per hour is: 33.3625
For m = 4 and s = 2, average cost per hour is: 30.525
For m = 4 and s = 3, average cost per hour is: 32.4875
For m = 4 and s = 4, average c

In [37]:
for m in m_values:
    for s in s_values:
        avg_cost = simulate_system(m, s, 800)
        aList = [ m, s, avg_cost]
        print(aList)



/tmp/ipykernel_42271/3750290773.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_breakdown_time[i] = exponential_distribution(8, 1)
/tmp/ipykernel_42271/3750290773.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_working_time[i] = exponential_distribution(8, 1)


[1, 1, 8.0375]
[1, 2, 8.9]
[1, 3, 7.5125]
[1, 4, 7.725]
[1, 5, 6.8875]
[2, 1, 17.0375]
[2, 2, 15.075]
[2, 3, 15.6375]
[2, 4, 16.2125]
[2, 5, 17.025]
[3, 1, 21.8875]
[3, 2, 26.125]
[3, 3, 25.275]
[3, 4, 23.825]
[3, 5, 25.25]
[4, 1, 31.525]
[4, 2, 31.3875]
[4, 3, 29.8]
[4, 4, 33.4]
[4, 5, 32.5875]
[5, 1, 40.3375]
[5, 2, 40.1]
[5, 3, 41.9875]
[5, 4, 38.8875]
[5, 5, 39.7625]
[6, 1, 46.65]
[6, 2, 47.0375]
[6, 3, 48.2375]
[6, 4, 49.275]
[6, 5, 46.95]
[7, 1, 54.4375]
[7, 2, 53.8625]
[7, 3, 54.2]
[7, 4, 56.075]
[7, 5, 56.3875]
[8, 1, 62.325]
[8, 2, 63.2875]
[8, 3, 61.9625]
[8, 4, 60.05]
[8, 5, 64.225]
[9, 1, 70.1375]
[9, 2, 67.95]
[9, 3, 70.4625]
[9, 4, 70.9875]
[9, 5, 74.2875]
[10, 1, 78.65]
[10, 2, 77.425]
[10, 3, 76.875]
[10, 4, 79.35]
[10, 5, 75.9875]
[11, 1, 89.025]
[11, 2, 88.85]
[11, 3, 84.9]
[11, 4, 85.2375]
[11, 5, 87.0625]
[12, 1, 96.6125]
[12, 2, 94.925]
[12, 3, 96.575]
[12, 4, 95.1625]
[12, 5, 90.3375]
[13, 1, 101.8125]
[13, 2, 99.4375]
[13, 3, 101.8875]
[13, 4, 102.65]
[13, 5, 101

In [38]:
for m in m_values:
    for s in s_values:
        avg_cost = simulate_system(m, s, 800)
        print( m, s, avg_cost)

/tmp/ipykernel_42271/3750290773.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_breakdown_time[i] = exponential_distribution(8, 1)
/tmp/ipykernel_42271/3750290773.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_working_time[i] = exponential_distribution(8, 1)


1 1 7.775
1 2 8.1875
1 3 8.0375
1 4 7.6125
1 5 7.725
2 1 14.8375
2 2 15.875
2 3 16.425
2 4 15.3
2 5 15.675
3 1 23.3625
3 2 22.7
3 3 24.1125
3 4 23.375
3 5 23.85
4 1 33.225
4 2 32.6875
4 3 32.5625
4 4 31.5625
4 5 31.45
5 1 38.6375
5 2 39.5875
5 3 40.725
5 4 38.7875
5 5 38.2875
6 1 47.7625
6 2 47.375
6 3 47.55
6 4 46.2125
6 5 47.4375
7 1 57.0
7 2 54.0875
7 3 53.825
7 4 56.4125
7 5 55.3875
8 1 64.125
8 2 65.6
8 3 65.575
8 4 67.85
8 5 65.0
9 1 72.5
9 2 72.025
9 3 73.125
9 4 70.5375
9 5 71.6
10 1 77.6875
10 2 79.85
10 3 81.5375
10 4 79.6125
10 5 79.4375
11 1 86.975
11 2 87.95
11 3 87.3
11 4 88.275
11 5 88.6375
12 1 98.4875
12 2 95.6625
12 3 94.7375
12 4 96.975
12 5 91.6125
13 1 102.2
13 2 100.8
13 3 100.775
13 4 103.7625
13 5 103.975
14 1 112.325
14 2 104.8875
14 3 110.05
14 4 108.525
14 5 109.0
15 1 118.675
15 2 121.9625
15 3 118.125
15 4 117.875
15 5 117.35
16 1 124.1625
16 2 125.325
16 3 123.4125
16 4 126.0625
16 5 126.7625
17 1 136.375
17 2 133.45
17 3 133.025
17 4 135.525
17 5 137.4125